# GraphNeT Baseline Submission
<img style="float: right;" src="https://raw.githubusercontent.com/graphnet-team/graphnet/main/assets/identity/graphnet-logo-and-wordmark.png" width="600" height="600" />

This notebook submits predictions from the public pre-trained dynedge to the leaderboard. 

In [4]:
# Move software to working disk
!rm  -r software
!scp -r /kaggle/input/graphnet-and-dependencies/software .

# Install dependencies
!pip install /kaggle/working/software/dependencies/torch-1.11.0+cu115-cp37-cp37m-linux_x86_64.whl
!pip install /kaggle/working/software/dependencies/torch_cluster-1.6.0-cp37-cp37m-linux_x86_64.whl
!pip install /kaggle/working/software/dependencies/torch_scatter-2.0.9-cp37-cp37m-linux_x86_64.whl
!pip install /kaggle/working/software/dependencies/torch_sparse-0.6.13-cp37-cp37m-linux_x86_64.whl
!pip install /kaggle/working/software/dependencies/torch_geometric-2.0.4.tar.gz

# Install GraphNeT
!cd software/graphnet;pip install --no-index --find-links="/kaggle/working/software/dependencies" -e .[torch]

# Append to PATH
import sys
sys.path.append('/kaggle/working/software/graphnet/src')

Processing ./software/dependencies/torch-1.11.0+cu115-cp37-cp37m-linux_x86_64.whl
torch is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.
Processing ./software/dependencies/torch_cluster-1.6.0-cp37-cp37m-linux_x86_64.whl
torch-cluster is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.
Processing ./software/dependencies/torch_scatter-2.0.9-cp37-cp37m-linux_x86_64.whl
torch-scatter is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.
Processing ./software/dependencies/torch_sparse-0.6.13-cp37-cp37m-linux_x86_64.whl
torch-sparse is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.
Processing ./software/dependencies/torch_geometric-2.0.4.tar.gz
  Preparing metadata (setup.py) ... done
  Created

In [30]:
import pyarrow.parquet as pq
import sqlite3
import pandas as pd
import sqlalchemy
from tqdm import tqdm
import os
from typing import Any, Dict, List, Optional
import numpy as np

from graphnet.data.sqlite.sqlite_utilities import create_table

def load_input(meta_batch: pd.DataFrame, input_data_folder: str) -> pd.DataFrame:
        """
        Will load the corresponding detector readings associated with the meta data batch.
        """
        batch_id = pd.unique(meta_batch['batch_id'])

        assert len(batch_id) == 1, "contains multiple batch_ids. Did you set the batch_size correctly?"
        
        detector_readings = pd.read_parquet(path = f'{input_data_folder}/batch_{batch_id[0]}.parquet')
        sensor_positions = geometry_table.loc[detector_readings['sensor_id'], ['x', 'y', 'z']]
        sensor_positions.index = detector_readings.index

        for column in sensor_positions.columns:
            if column not in detector_readings.columns:
                detector_readings[column] = sensor_positions[column]

        detector_readings['auxiliary'] = detector_readings['auxiliary'].replace({True: 1, False: 0})
        return detector_readings.reset_index()

def add_to_table(database_path: str,
                      df: pd.DataFrame,
                      table_name:  str,
                      is_primary_key: bool,
                      ) -> None:
    """Writes meta data to sqlite table. 

    Args:
        database_path (str): the path to the database file.
        df (pd.DataFrame): the dataframe that is being written to table.
        table_name (str, optional): The name of the meta table. Defaults to 'meta_table'.
        is_primary_key(bool): Must be True if each row of df corresponds to a unique event_id. Defaults to False.
    """
    try:
        print(database_path)
        create_table(   columns=  df.columns,
                        database_path = database_path, 
                        table_name = table_name,
                        integer_primary_key= is_primary_key,
                        index_column = 'event_id')
    except sqlite3.OperationalError as e:
        if 'already exists' in str(e):
            pass
        else:
            raise e
    engine = sqlalchemy.create_engine("sqlite:///" + database_path)
    df.to_sql(table_name, con=engine, index=False, if_exists="append", chunksize = 200000)
    engine.dispose()
    return

def convert_to_sqlite(meta_data_path: str,
                      database_path: str,
                      input_data_folder: str,
                      batch_size: int = 200000) -> None:
    """Converts a selection of the Competition's parquet files to a single sqlite database.

    Args:
        meta_data_path (str): Path to the meta data file.
        batch_size (int): the number of rows extracted from meta data file at a time. Keep low for memory efficiency.
        database_path (str): path to database. E.g. '/my_folder/data/my_new_database.db'
        input_data_folder (str): folder containing the parquet input files.
        accepted_batch_ids (List[int]): The batch_ids you want converted. Defaults to None (all batches will be converted)
    """
    meta_data_iter = pq.ParquetFile(meta_data_path).iter_batches(batch_size = batch_size)
    
    if not database_path.endswith('.db'):
        database_path = database_path+'.db'
        
    converted_batches = [] 
    progress_bar = tqdm(total = None)
    for meta_data_batch in meta_data_iter:
        unique_batch_ids = pd.unique(meta_data_batch['event_id']).tolist()
        meta_data_batch  = meta_data_batch.to_pandas()
        add_to_table(database_path = database_path,
                    df = meta_data_batch,
                    table_name='meta_table',
                    is_primary_key= True)
        pulses = load_input(meta_batch=meta_data_batch, input_data_folder= input_data_folder)
        del meta_data_batch # memory
        add_to_table(database_path = database_path,
                    df = pulses,
                    table_name='pulse_table',
                    is_primary_key= False)
        del pulses # memory
        progress_bar.update(1)
    progress_bar.close()
    del meta_data_iter # memory
    print(f'Conversion Complete!. Database available at\n {database_path}')

In [31]:
!rm '/kaggle/working/test_database.db'
input_data_folder = '/kaggle/input/icecube-neutrinos-in-deep-ice/test'
geometry_table = pd.read_csv('/kaggle/input/icecube-neutrinos-in-deep-ice/sensor_geometry.csv')
meta_data_path = '/kaggle/input/icecube-neutrinos-in-deep-ice/test_meta.parquet'

database_path = '/kaggle/working/test_database'
convert_to_sqlite(meta_data_path,
                  database_path=database_path,
                  input_data_folder=input_data_folder)

0it [00:00, ?it/s]

/kaggle/working/test_database.db
/kaggle/working/test_database.db


1it [00:00, 10.82it/s]

Conversion Complete!. Database available at
 /kaggle/working/test_database.db


In [32]:
from pytorch_lightning.callbacks import EarlyStopping
from torch.optim.adam import Adam
from graphnet.data.constants import FEATURES, TRUTH
from graphnet.models import StandardModel
from graphnet.models.detector.icecube import IceCubeKaggle
from graphnet.models.gnn import DynEdge
from graphnet.models.graph_builders import KNNGraphBuilder
from graphnet.models.task.reconstruction import DirectionReconstructionWithKappa, ZenithReconstructionWithKappa, AzimuthReconstructionWithKappa
from graphnet.training.callbacks import ProgressBar, PiecewiseLinearLR
from graphnet.training.loss_functions import VonMisesFisher3DLoss, VonMisesFisher2DLoss
from graphnet.training.labels import Direction
from graphnet.training.utils import make_dataloader
from graphnet.utilities.logging import get_logger
from pytorch_lightning import Trainer
import pandas as pd

logger = get_logger()

def build_model(config: Dict[str,Any], train_dataloader: Any) -> StandardModel:
    """Builds GNN from config"""
    # Building model
    detector = IceCubeKaggle(
        graph_builder=KNNGraphBuilder(nb_nearest_neighbours=8),
    )
    gnn = DynEdge(
        nb_inputs=detector.nb_outputs,
        global_pooling_schemes=["min", "max", "mean"],
    )

    if config["target"] == 'direction':
        task = DirectionReconstructionWithKappa(
            hidden_size=gnn.nb_outputs,
            target_labels=config["target"],
            loss_function=VonMisesFisher3DLoss(),
        )
        prediction_columns = [config["target"] + "_x", 
                              config["target"] + "_y", 
                              config["target"] + "_z", 
                              config["target"] + "_kappa" ]
        additional_attributes = ['zenith', 'azimuth', 'event_id']

    model = StandardModel(
        detector=detector,
        gnn=gnn,
        tasks=[task],
        optimizer_class=Adam,
        optimizer_kwargs={"lr": 5e-04, "eps": 5e-04},
        scheduler_class=PiecewiseLinearLR,
        scheduler_kwargs={
            "milestones": [
                0,
                len(train_dataloader) / 2,
                len(train_dataloader) * config["fit"]["max_epochs"],
            ],
            "factors": [1e-02, 1, 1e-02],
        },
        scheduler_config={
            "interval": "step",
        },
    )
    model.prediction_columns = prediction_columns
    model.additional_attributes = additional_attributes
    
    return model

def build_model1(config: Dict[str,Any], train_dataloader: Any) -> StandardModel:
    """Builds GNN from config"""
    # Building model
    detector = IceCubeKaggle(
        graph_builder=KNNGraphBuilder(nb_nearest_neighbours=12),
    )
    gnn = DynEdge(
        nb_inputs=detector.nb_outputs,
        global_pooling_schemes=["min", "max", "mean"],
    )

    if config["target"] == 'direction':
        task = DirectionReconstructionWithKappa(
            hidden_size=gnn.nb_outputs,
            target_labels=config["target"],
            loss_function=VonMisesFisher3DLoss(),
        )
        prediction_columns = [config["target"] + "_x", 
                              config["target"] + "_y", 
                              config["target"] + "_z", 
                              config["target"] + "_kappa" ]
        additional_attributes = ['zenith', 'azimuth', 'event_id']

    model = StandardModel(
        detector=detector,
        gnn=gnn,
        tasks=[task],
        optimizer_class=Adam,
        optimizer_kwargs={"lr": 5e-04, "eps": 5e-04},
        scheduler_class=PiecewiseLinearLR,
        scheduler_kwargs={
            "milestones": [
                0,
                len(train_dataloader) / 2,
                len(train_dataloader) * config["fit"]["max_epochs"],
            ],
            "factors": [1e-02, 1, 1e-02],
        },
        scheduler_config={
            "interval": "step",
        },
    )
    model.prediction_columns = prediction_columns
    model.additional_attributes = additional_attributes
    
    return model


def load_pretrained_model(config: Dict[str,Any], state_dict_path: str = '/kaggle/input/dynedge-pretrained/dynedge_pretrained_batch_1_to_50/state_dict.pth') -> StandardModel:
    train_dataloader, _ = make_dataloaders(config = config)
    model = build_model(config = config, 
                        train_dataloader = train_dataloader)
    #model._inference_trainer = Trainer(config['fit'])
    model.load_state_dict(state_dict_path)
    model.prediction_columns = [config["target"] + "_x", 
                              config["target"] + "_y", 
                              config["target"] + "_z", 
                              config["target"] + "_kappa" ]
    model.additional_attributes = ['event_id'] #'zenith', 'azimuth',  not available in test data
    return model

def load_pretrained_model2(config: Dict[str,Any], state_dict_path: str = '/kaggle/input/icecubemodel/model(1).pth') -> StandardModel:
    train_dataloader, _ = make_dataloaders(config = config)
    model = build_model1(config = config, 
                        train_dataloader = train_dataloader)
    #model._inference_trainer = Trainer(config['fit'])
    model.load_state_dict(state_dict_path)
    model.prediction_columns = [config["target"] + "_x", 
                              config["target"] + "_y", 
                              config["target"] + "_z", 
                              config["target"] + "_kappa" ]
    model.additional_attributes = ['event_id'] #'zenith', 'azimuth',  not available in test data
    return model


def make_dataloaders(config: Dict[str, Any]) -> List[Any]:
    """Constructs training and validation dataloaders for training with early stopping."""
    
    train_dataloader = make_dataloader(db = config['path'],
                                            selection = pd.read_csv(config['train_selection'])[config['index_column']].ravel().tolist() if config['train_selection'] else None,
                                            pulsemaps = config['pulsemap'],
                                            features = features,
                                            truth = truth,
                                            batch_size = config['batch_size'],
                                            num_workers = config['num_workers'],
                                            shuffle = True,
                                            labels = {'direction': Direction()},
                                            index_column = config['index_column'],
                                            truth_table = config['truth_table'],
                                            )
    
    validate_dataloader = make_dataloader(db = config['path'],
                                            selection = pd.read_csv(config['validate_selection'])[config['index_column']].ravel().tolist() if config['validate_selection'] else None,
                                            pulsemaps = config['pulsemap'],
                                            features = features,
                                            truth = truth,
                                            batch_size = config['batch_size'],
                                            num_workers = config['num_workers'],
                                            shuffle = False,
                                            labels = {'direction': Direction()},
                                            index_column = config['index_column'],
                                            truth_table = config['truth_table'],
                                          
                                            )
    return train_dataloader, validate_dataloader

def inference(models, config: Dict[str, Any]) -> pd.DataFrame:
    """Applies model to the database specified in config['inference_database_path'] and saves results to disk."""
    # Make Dataloader
    test_dataloader = make_dataloader(db = config['inference_database_path'],
                                            selection = None, # Entire database
                                            pulsemaps = config['pulsemap'],
                                            features = features,
                                            truth = truth,
                                            batch_size = config['batch_size'],
                                            num_workers = config['num_workers'],
                                            shuffle = False,
                                            labels = None, # Cannot make labels in test data
                                            index_column = config['index_column'],
                                            truth_table = config['truth_table'],
                                            )
    
    # Get predictions
    results1 = models[0].predict_as_dataframe(
        gpus = [0],
        dataloader = test_dataloader,
        prediction_columns=models[0].prediction_columns,
        additional_attributes=['event_id']
    )
    
    print(results1)
    
    results2 = models[1].predict_as_dataframe(
        gpus = [0],
        dataloader = test_dataloader,
        prediction_columns=models[1].prediction_columns,
        additional_attributes=['event_id']
    )

    results1["direction_x"] = 0.9 * results1["direction_x"] + 0.1 * results2["direction_x"]
    results1["direction_y"] = 0.9 * results1["direction_y"] + 0.1 * results2["direction_y"]
    results1["direction_z"] = 0.9 * results1["direction_z"] + 0.1 * results2["direction_z"]
    results1["direction_kappa"] = 0.9 * results1["direction_kappa"] + 0.1 * results2["direction_kappa"]

    return results1

    

In [33]:
def prepare_dataframe(df, angle_post_fix = '_reco', vec_post_fix = '') -> pd.DataFrame:
    r = np.sqrt(df['direction_x'+ vec_post_fix]**2 + df['direction_y'+ vec_post_fix]**2 + df['direction_z' + vec_post_fix]**2)
    df['zenith' + angle_post_fix] = np.arccos(df['direction_z'+ vec_post_fix]/r)
    df['azimuth'+ angle_post_fix] = np.arctan2(df['direction_y'+ vec_post_fix],df['direction_x' + vec_post_fix]) #np.sign(results['true_y'])*np.arccos((results['true_x'])/(np.sqrt(results['true_x']**2 + results['true_y']**2)))
    df['azimuth'+ angle_post_fix][df['azimuth'  + angle_post_fix]<0] = df['azimuth'  + angle_post_fix][df['azimuth'  +  angle_post_fix]<0] + 2*np.pi 

    drop_these_columns = []
    for column in results.columns:
        if column not in ['event_id', 'zenith', 'azimuth']:
            drop_these_columns.append(column)
    return df.drop(columns = drop_these_columns).iloc[:,[0,2,1]].set_index('event_id')
    


In [34]:
# Constants
features = FEATURES.KAGGLE
truth = TRUTH.KAGGLE

# Configuration
config = {
        "path": '/kaggle/working/test_database.db',
        "inference_database_path": '/kaggle/working/test_database.db',
        "pulsemap": 'pulse_table',
        "truth_table": 'meta_table',
        "features": features,
        "truth": truth,
        "index_column": 'event_id',
        "run_name_tag": 'graphnet_baseline_submission',
        "batch_size": 250,
        "num_workers": 2,
        "target": 'direction',
        "early_stopping_patience": 5,
        "fit": {
                "max_epochs": 70,
                "gpus": [0],
                "distribution_strategy": None,
                },
        'train_selection': None,
        'validate_selection':  None,
        'test_selection': None,
        'base_dir': 'training'
}
models = [load_pretrained_model(config = config), load_pretrained_model(config = config)]

results = inference(models, config)

graphnet: WARNING  2023-04-16 04:52:20 - SQLiteDataset.warning - Removing the following (missing) truth variables: zenith, azimuth
graphnet: WARNING  2023-04-16 04:52:20 - SQLiteDataset.warning - Removing the following (missing) truth variables: zenith, azimuth
graphnet: WARNING  2023-04-16 04:52:20 - SQLiteDataset.warning - Removing the following (missing) truth variables: zenith, azimuth
graphnet: WARNING  2023-04-16 04:52:20 - SQLiteDataset.warning - Removing the following (missing) truth variables: zenith, azimuth
graphnet: WARNING  2023-04-16 04:52:20 - SQLiteDataset.warning - Removing the following (missing) truth variables: zenith, azimuth


Predicting: 0it [00:00, ?it/s]

   direction_x  direction_y  direction_z  direction_kappa  event_id
0     0.987893    -0.148705     0.044196         2.946704    2092.0
1    -0.551074    -0.254085    -0.794832         6.980600    7344.0
2    -0.164367    -0.986232     0.018184        86.766502    9482.0


Predicting: 0it [00:00, ?it/s]

In [35]:
results

,direction_x,direction_y,direction_z,direction_kappa,event_id
0,0.987893,-0.148705,0.044196,2.946704,2092.0
1,-0.551074,-0.254085,-0.794832,6.980601,7344.0
2,-0.164367,-0.986232,0.018184,86.766502,9482.0


In [36]:
submission_df = prepare_dataframe(results, angle_post_fix = '')

In [37]:
#submission_df.to_csv('submission.csv')

In [38]:
df_1 = submission_df

In [40]:
del models
del results
del submission_df

In [39]:
gc.collect()



173297

In [14]:

# Import Modules
import gc
import os
import multiprocessing
import time
import numpy as np
import pandas as pd
import pyarrow.parquet as pq
import tensorflow as tf
from tqdm import tqdm

5787

In [16]:
# Directories and constants
home_dir = "/kaggle/input/icecube-neutrinos-in-deep-ice/"
test_format = home_dir + 'test/batch_{batch_id:d}.parquet'
model_home = "/kaggle/input/lstmicecubesdata/"

# Model(s)
model_names = ["4347_MAE_1-02076_bin24_pp96_n6_batch2048_epoch29.h5",
               "4347_MAE_1-02039_bin24_pp96_n6_batch2048_epoch25.h5", 
               "4346_MAE_1-02020_bin24_pp96_n6_batch2048_epoch27.h5"]
model_weights = np.array([0.30, 
                          0.30,
                          0.40])

In [17]:
# Load Models
models = []
for model_name in model_names:
    print(f'\n========== Model File: {model_name}')
    
    # Load Model
    model_path = model_home + model_name
    model = tf.keras.models.load_model(model_path, compile = False)
    models.append(model)      
    
    # Model summary
    model.summary()
    
# Get Model Parameters
pulse_count = model.inputs[0].shape[1]
feature_count = model.inputs[0].shape[2]
output_bins = model.layers[-1].weights[0].shape[-1]
bin_num = int(np.sqrt(output_bins))

# Model Parameter Summary
print("\n==== Model Parameters")
print(f"Bin Numbers: {bin_num}")
print(f"Maximum Pulse Count: {pulse_count}")
print(f"Features Count: {feature_count}")


========== Model File: 4347_MAE_1-02076_bin24_pp96_n6_batch2048_epoch29.h5
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 96, 6)]           0         
_________________________________________________________________
masking (Masking)            (None, 96, 6)             0         
_________________________________________________________________
bidirectional (Bidirectional (None, 96, 384)           230400    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 96, 384)           665856    
_________________________________________________________________
bidirectional_2 (Bidirection (None, 384)               665856    
_________________________________________________________________
dense (Dense)                (None, 256)               98560     
___________________________________________________

Exception in thread QueueFeederThread:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/multiprocessing/queues.py", line 232, in _feed
    close()
  File "/opt/conda/lib/python3.7/multiprocessing/connection.py", line 177, in close
    self._close()
  File "/opt/conda/lib/python3.7/multiprocessing/connection.py", line 361, in _close
    _close(self._handle)
OSError: [Errno 9] Bad file descriptor

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/opt/conda/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.7/multiprocessing/queues.py", line 263, in _feed
    queue_sem.release()
ValueError: semaphore or lock released too many times

Exception in thread QueueFeederThread:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/multipro

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 96, 6)]           0         
_________________________________________________________________
masking (Masking)            (None, 96, 6)             0         
_________________________________________________________________
bidirectional (Bidirectional (None, 96, 384)           230400    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 96, 384)           665856    
_________________________________________________________________
bidirectional_2 (Bidirection (None, 384)               665856    
_________________________________________________________________
dense (Dense)                (None, 256)               98560     
_________________________________________________________________
dense_1 (Dense)              (None, 576)               148032

In [18]:
# Load sensor_geometry
sensor_geometry_df = pd.read_csv(home_dir + "sensor_geometry.csv")

# Get Sensor Information
sensor_x = sensor_geometry_df.x
sensor_y = sensor_geometry_df.y
sensor_z = sensor_geometry_df.z

# Detector constants
c_const = 0.299792458  # speed of light [m/ns]

# Sensor Min / Max Coordinates
x_min = sensor_x.min()
x_max = sensor_x.max()
y_min = sensor_y.min()
y_max = sensor_y.max()
z_min = sensor_z.min()
z_max = sensor_z.max()

detector_length = np.sqrt((x_max - x_min)**2 + (y_max - y_min)**2 + (z_max - z_min)**2)
t_valid_length = detector_length / c_const

print(f"time valid length: {t_valid_length} ns")

time valid length: 6199.700247193777 ns


In [19]:
# Create Azimuth Edges
azimuth_edges = np.linspace(0, 2 * np.pi, bin_num + 1)
print(azimuth_edges)

# Create Zenith Edges
zenith_edges = []
zenith_edges.append(0)
for bin_idx in range(1, bin_num):
    zenith_edges.append(np.arccos(np.cos(zenith_edges[-1]) - 2 / (bin_num)))
zenith_edges.append(np.pi)
zenith_edges = np.array(zenith_edges)
print(zenith_edges)

[0.         0.26179939 0.52359878 0.78539816 1.04719755 1.30899694
 1.57079633 1.83259571 2.0943951  2.35619449 2.61799388 2.87979327
 3.14159265 3.40339204 3.66519143 3.92699082 4.1887902  4.45058959
 4.71238898 4.97418837 5.23598776 5.49778714 5.75958653 6.02138592
 6.28318531]
[0.         0.41113786 0.58568554 0.72273425 0.84106867 0.94796974
 1.04719755 1.1410209  1.23095942 1.31811607 1.40334825 1.48736624
 1.57079633 1.65422641 1.73824441 1.82347658 1.91063324 2.00057176
 2.0943951  2.19362291 2.30052398 2.41885841 2.55590711 2.73045479
 3.14159265]


In [20]:
angle_bin_zenith0 = np.tile(zenith_edges[:-1], bin_num)
angle_bin_zenith1 = np.tile(zenith_edges[1:], bin_num)
angle_bin_azimuth0 = np.repeat(azimuth_edges[:-1], bin_num)
angle_bin_azimuth1 = np.repeat(azimuth_edges[1:], bin_num)

angle_bin_area = (angle_bin_azimuth1 - angle_bin_azimuth0) * (np.cos(angle_bin_zenith0) - np.cos(angle_bin_zenith1))
angle_bin_vector_sum_x = (np.sin(angle_bin_azimuth1) - np.sin(angle_bin_azimuth0)) * ((angle_bin_zenith1 - angle_bin_zenith0) / 2 - (np.sin(2 * angle_bin_zenith1) - np.sin(2 * angle_bin_zenith0)) / 4)
angle_bin_vector_sum_y = (np.cos(angle_bin_azimuth0) - np.cos(angle_bin_azimuth1)) * ((angle_bin_zenith1 - angle_bin_zenith0) / 2 - (np.sin(2 * angle_bin_zenith1) - np.sin(2 * angle_bin_zenith0)) / 4)
angle_bin_vector_sum_z = (angle_bin_azimuth1 - angle_bin_azimuth0) * ((np.cos(2 * angle_bin_zenith0) - np.cos(2 * angle_bin_zenith1)) / 4)

angle_bin_vector_mean_x = angle_bin_vector_sum_x / angle_bin_area
angle_bin_vector_mean_y = angle_bin_vector_sum_y / angle_bin_area
angle_bin_vector_mean_z = angle_bin_vector_sum_z / angle_bin_area

angle_bin_vector = np.zeros((1, bin_num * bin_num, 3))
angle_bin_vector[:, :, 0] = angle_bin_vector_mean_x
angle_bin_vector[:, :, 1] = angle_bin_vector_mean_y
angle_bin_vector[:, :, 2] = angle_bin_vector_mean_z

angle_bin_vector_unit = angle_bin_vector[0].copy()
angle_bin_vector_unit /= np.sqrt((angle_bin_vector_unit**2).sum(axis=1).reshape((-1, 1)))

In [21]:
def pred_to_angle(pred, epsilon = 1e-8):
    # Convert prediction
    pred_vector = (pred.reshape((-1, bin_num**2, 1)) * angle_bin_vector).sum(axis = 1)
    
    # Normalize
    pred_vector_norm = np.sqrt((pred_vector**2).sum(axis = 1))
    mask = pred_vector_norm < epsilon
    pred_vector_norm[mask] = 1
    
    # Assign <1, 0, 0> to very small vectors (badly predicted)
    pred_vector /= pred_vector_norm.reshape((-1, 1))
    pred_vector[mask] = np.array([1., 0., 0.])
    
    # Convert to angle
    azimuth = np.arctan2(pred_vector[:, 1], pred_vector[:, 0])
    azimuth[azimuth < 0] += 2 * np.pi
    zenith = np.arccos(pred_vector[:, 2])
    
    # Mask bad norm predictions as 0, 0
    azimuth[mask] = 0.
    zenith[mask] = 0.
    
    return azimuth, zenith

In [22]:
def weighted_vector_ensemble(angles, weight):
    # Convert angle to vector
    vec_models = list()
    for angle in angles:
        az, zen = angle
        sa = np.sin(az)
        ca = np.cos(az)
        sz = np.sin(zen)
        cz = np.cos(zen)
        vec = np.stack([sz * ca, sz * sa, cz], axis=1)
        vec_models.append(vec)
    vec_models = np.array(vec_models)

    # Weighted-mean
    vec_mean = (weight.reshape((-1, 1, 1)) * vec_models).sum(axis=0) / weight.sum()
    vec_mean /= np.sqrt((vec_mean**2).sum(axis=1)).reshape((-1, 1))

    # Convert vector to angle
    zenith = np.arccos(vec_mean[:, 2])
    azimuth = np.arctan2(vec_mean[:, 1], vec_mean[:, 0])
    azimuth[azimuth < 0] += 2 * np.pi
    
    return azimuth, zenith

In [23]:
# Placeholder
open_batch_dict = dict()

# Read single event from batch_meta_df
def read_event(event_idx, batch_meta_df, pulse_count):
    # Read metadata
    batch_id, first_pulse_index, last_pulse_index = batch_meta_df.iloc[event_idx][["batch_id", "first_pulse_index", "last_pulse_index"]].astype("int")

    # close past batch df
    if batch_id - 1 in open_batch_dict.keys():
        del open_batch_dict[batch_id - 1]

    # open current batch df
    if batch_id not in open_batch_dict.keys():
        open_batch_dict.update({batch_id: pd.read_parquet(test_format.format(batch_id=batch_id))})
    
    batch_df = open_batch_dict[batch_id]
    
    # Read event
    event_feature = batch_df[first_pulse_index:last_pulse_index + 1]
    sensor_id = event_feature.sensor_id
    
    # Merge features into single structured array
    dtype = [("time", "float16"),
             ("charge", "float16"),
             ("auxiliary", "float16"),
             ("x", "float16"),
             ("y", "float16"),
             ("z", "float16"),
             ("rank", "short")]    
    
    # Create event_x
    event_x = np.zeros(last_pulse_index - first_pulse_index + 1, dtype)
    event_x["time"] = event_feature.time.values - event_feature.time.min()
    event_x["charge"] = event_feature.charge.values
    event_x["auxiliary"] = event_feature.auxiliary.values
    event_x["x"] = sensor_geometry_df.x[sensor_id].values
    event_x["y"] = sensor_geometry_df.y[sensor_id].values
    event_x["z"] = sensor_geometry_df.z[sensor_id].values

    # For long event, pick-up
    if len(event_x) > pulse_count:
        # Find valid time window
        t_peak = event_x["time"][event_x["charge"].argmax()]
        t_valid_min = t_peak - t_valid_length
        t_valid_max = t_peak + t_valid_length
        t_valid = (event_x["time"] > t_valid_min) * (event_x["time"] < t_valid_max)

        # Rank
        event_x["rank"] = 2 * (1 - event_x["auxiliary"]) + (t_valid)

        # Sort by Rank and Charge (important goes to backward)
        event_x = np.sort(event_x, order = ["rank", "charge"])

        # pick-up from backward
        event_x = event_x[-pulse_count:]

        # Sort events by time 
        event_x = np.sort(event_x, order = "time")

    return event_idx, len(event_x), event_x

In [24]:
# Read Test Meta data
test_meta_df = pq.read_table(home_dir + 'test_meta.parquet').to_pandas()
batch_counts = test_meta_df.batch_id.value_counts().sort_index()

batch_max_index = batch_counts.cumsum()
batch_max_index[test_meta_df.batch_id.min() - 1] = 0
batch_max_index = batch_max_index.sort_index()

# Support Function
def test_meta_df_spliter(batch_id):
    return test_meta_df.loc[batch_max_index[batch_id - 1]:batch_max_index[batch_id] - 1]

In [25]:
# Get Batch IDs
test_batch_ids = test_meta_df.batch_id.unique()

# Submission Placeholders
test_event_id = []
test_azimuth = []
test_zenith = []

# Batch Loop
for batch_id in test_batch_ids:
    # Batch Meta DF
    batch_meta_df = test_meta_df_spliter(batch_id)

    # Set Pulses
    test_x = np.zeros((len(batch_meta_df), pulse_count, feature_count), dtype = "float16")    
    test_x[:, :, 2] = -1    

    # Read Event Data
    def read_event_local(event_idx):
        return read_event(event_idx, batch_meta_df, pulse_count)
    
    # Multiprocess Events
    iterator = range(len(batch_meta_df))
    with multiprocessing.Pool() as pool:
        for event_idx, pulsecount, event_x in pool.map(read_event_local, iterator):
            # Features
            test_x[event_idx, :pulsecount, 0] = event_x["time"]
            test_x[event_idx, :pulsecount, 1] = event_x["charge"]
            test_x[event_idx, :pulsecount, 2] = event_x["auxiliary"]
            test_x[event_idx, :pulsecount, 3] = event_x["x"]
            test_x[event_idx, :pulsecount, 4] = event_x["y"]
            test_x[event_idx, :pulsecount, 5] = event_x["z"]
    
    del batch_meta_df
    
    # Normalize
    test_x[:, :, 0] /= 1000  # time
    test_x[:, :, 1] /= 300  # charge
    test_x[:, :, 3:] /= 600  # space
        
    # Predict
    pred_angles = []
    for model in models:
        pred_model = model.predict(test_x, verbose=0)
        az_model, zen_model = pred_to_angle(pred_model)
        pred_angles.append((az_model, zen_model))
    
    # Get Predicted Azimuth and Zenith
    pred_azimuth, pred_zenith = weighted_vector_ensemble(pred_angles, model_weights)
    
    # Get Event IDs
    event_ids = test_meta_df.event_id[test_meta_df.batch_id == batch_id].values
    
    # Finalize 
    for event_id, azimuth, zenith in zip(event_ids, pred_azimuth, pred_zenith):
        if np.isfinite(azimuth) and np.isfinite(zenith):
            test_event_id.append(int(event_id))
            test_azimuth.append(azimuth)
            test_zenith.append(zenith)
        else:
            test_event_id.append(int(event_id))
            test_azimuth.append(0.)
            test_zenith.append(0.)

Could not load symbol cublasGetSmCountTarget from libcublas.so.11. Error: /opt/conda/bin/../lib/libcublas.so.11: undefined symbol: cublasGetSmCountTarget


In [26]:
# Create and Save Submission.csv
submission_df = pd.DataFrame({"event_id": test_event_id,
                              "azimuth": test_azimuth,
                              "zenith": test_zenith})
submission_df = submission_df.sort_values(by = ['event_id'])
#submission_df.to_csv("submission.csv", index = False)

In [27]:
df_2 = submission_df

In [28]:
# Merge the two dataframes using the event_id column key
merged_df = pd.merge(df_1, df_2, on='event_id')

#weights of the ensamblers
prop_1 = 0.1
prop_2 = 0.9

# Compute the weighted average of azimuth and zenith columns
merged_df['azimuth'] = (prop_1 * merged_df['azimuth_x']) + (prop_2 * merged_df['azimuth_y'])
merged_df['zenith'] = (prop_1 * merged_df['zenith_x']) + (prop_2 * merged_df['zenith_y'])

# Drop the unnecessary columns
merged_df.drop(['azimuth_x', 'azimuth_y', 'zenith_x', 'zenith_y'], axis=1, inplace=True)

# Write the merged dataframe to a new CSV file
merged_df.to_csv('merged.csv', index=False)

In [29]:
merged_df.to_csv("submission.csv", index = False)